# Initialize a SparkSession

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('walmart').getOrCreate()

22/11/10 21:42:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Load the Walmart Data

In [2]:
df = spark.read.csv('./WMT.xls', inferSchema=True, header=True)

# Display the Column Names

In [3]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Display the first row of the dataframe

In [4]:
df.head()

Row(Date=datetime.datetime(2011, 11, 16, 0, 0), Open=57.099998, High=57.419998, Low=56.639999, Close=56.68, Adj Close=44.899456, Volume=11780800)

# Check the Schema of the Dataframe

In [5]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



# Print the first 5 rows

In [6]:
for line in df.head(5):
    print(line, '\n')

Row(Date=datetime.datetime(2011, 11, 16, 0, 0), Open=57.099998, High=57.419998, Low=56.639999, Close=56.68, Adj Close=44.899456, Volume=11780800) 

Row(Date=datetime.datetime(2011, 11, 17, 0, 0), Open=56.540001, High=57.189999, Low=56.259998, Close=56.73, Adj Close=44.939064, Volume=10223800) 

Row(Date=datetime.datetime(2011, 11, 18, 0, 0), Open=57.029999, High=57.360001, Low=56.610001, Close=57.23, Adj Close=45.335129, Volume=8982300) 

Row(Date=datetime.datetime(2011, 11, 21, 0, 0), Open=56.93, High=57.290001, Low=56.380001, Close=56.66, Adj Close=44.883606, Volume=9932200) 

Row(Date=datetime.datetime(2011, 11, 22, 0, 0), Open=56.560001, High=57.130001, Low=56.5, Close=56.849998, Adj Close=45.034107, Volume=7497300) 



# Print general Stats

In [7]:
df.describe().show()

+-------+------------------+------------------+------------------+-----------------+------------------+----------------+
|summary|              Open|              High|               Low|            Close|         Adj Close|          Volume|
+-------+------------------+------------------+------------------+-----------------+------------------+----------------+
|  count|              2516|              2516|              2516|             2516|              2516|            2516|
|   mean| 90.25209059817178| 90.94377592090628| 89.61924488434002|90.28167732551657| 82.77843126589822|8267158.86327504|
| stddev|25.818932768463412|26.070702860968247|25.565153063174236|25.79883827640457|29.207787105719017|4587081.88545677|
|    min|         56.389999|              57.0|         56.259998|        56.419998|         44.867764|         2094900|
|    max|        153.600006|        153.660004|        151.660004|       152.789993|        151.449997|        80898100|
+-------+------------------+----

# Format Columns to show just 2 Decimal Places

In [8]:
from pyspark.sql.functions import format_number

summary = df.describe()
summary.select(summary['summary'],
               format_number(summary['Open'].cast('float'),2).alias('Open'),
               format_number(summary['High'].cast('float'),2).alias('High'),
               format_number(summary['Low'].cast('float'),2).alias('Low'),
               format_number(summary['Close'].cast('float'),2).alias('Close'),
               format_number(summary['Volume'].cast('int'),0).alias('Volume')
              ).show()

+-------+--------+--------+--------+--------+----------+
|summary|    Open|    High|     Low|   Close|    Volume|
+-------+--------+--------+--------+--------+----------+
|  count|2,516.00|2,516.00|2,516.00|2,516.00|     2,516|
|   mean|   90.25|   90.94|   89.62|   90.28| 8,267,158|
| stddev|   25.82|   26.07|   25.57|   25.80| 4,587,081|
|    min|   56.39|   57.00|   56.26|   56.42| 2,094,900|
|    max|  153.60|  153.66|  151.66|  152.79|80,898,100|
+-------+--------+--------+--------+--------+----------+



# Create a new Dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [ ]:
df_hv = df.withColumn('HV Ratio', df['High']/df['Volume']).select(['HV Ratio'])
df_hv.show()

# Which day has the peak high in price ?

In [ ]:
df.orderBy(df['High'].desc()).select(['Date']).head(1)[0]['Date']

# What is the mean of the "Close" column

In [ ]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

# What is the maximum and minimum value of the "Volumn" Column

In [ ]:
from pyspark.sql.functions import min, max
df.select(max('Volume'),min('Volume')).show()

# How many days did the stocks close lower than 60 dollars?

In [ ]:
df.filter(df['Close'] < 60).count()

# What percentage of the time was the "High" greater than 80 dollars?

In [ ]:
df.filter(df['High'] > 80).count() * 100/df.count()

# What is the Pearson correlation between "High" and "Volume"

In [ ]:
df.corr('High','Volume')

In [ ]:
from pyspark.sql.functions import corr
df.select(corr(df['High'], df['Volume'])).show()

# What is the max "High" per year?

In [ ]:
from pyspark.sql.functions import (dayofmonth, hour, 
                                   dayofyear, month,
                                   year, weekofyear,
                                   format_number, date_format)

year_df = df.withColumn('Year', year(df['Date']))

year_df.groupBy('Year').max()['Year', 'max(High)'].show()

# What is the average "Close" for each calender month?

In [ ]:
#Create a new column Month from existing Date Column
month_df = df.withColumn('Month', month(df['Date']))

#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()

#Sort by month
month_df = month_df.orderBy('Month')

#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Close)'].show()